## ドライブ接続、ライブラリインストールなど

In [1]:
!pip install tensorflow==2.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 45.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninstalling gast-0.5.4:
      Successfully uninstalled gast-0.5.4
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.2.0
    Uninstalling google-auth-oauthlib-1.2.0:
      Successfully un

## 設定

In [9]:
# ファイルパス関連
# dataset = '/content/drive/Othercomputers/My Mac/Documents/2_study/1_修士/3_副研究/yubimoji/point_history_normalised.csv'
# model_save_path = '/content/drive/MyDrive/Colab Notebooks/JSL/model/gesture_classifier.hdf5'
# tflite_save_path = '/content/drive/Othercomputers/My Mac/Documents/2_study/1_修士/3_副研究/yubimoji/model/keypoint_classifier.tflite'

# 掌長にて正規化
# dataset = '/content/drive/Othercomputers/My Mac/Documents/2_study/1_修士/3_副研究/yubimoji/point_history_normalised_palm.csv'


# 通常の正規化
dataset = '/content/drive/Othercomputers/My Mac/Documents/2_study/1_修士/3_副研究/yubimoji/point_history_normalised_combined.csv'
model_save_path = '/content/drive/MyDrive/Colab Notebooks/JSL/palm_model/gesture_classifier.hdf5'
tflite_save_path = '/content/drive/Othercomputers/My Mac/Documents/2_study/1_修士/3_副研究/yubimoji/model/keypoint_classifier.tflite'

In [10]:
import csv
import numpy as np
import tensorflow as tf
from keras.utils import to_categorical

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

# モデルのパラメータ
n_features = 40  # ランドマークの数
n_sequence = 15  # シーケンス長
n_classes = 76 # 出力クラス (指文字) の数

# 2列目以降を学習データとする
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, n_features+1))) # (20730, 40)

# 1列目を正解ラベルとする
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

# ラベルをワンホットエンコードする
y_dataset = to_categorical(y_dataset, num_classes=n_classes)

# サンプル数をシーケンス長で割り切れるように調整
total_samples = len(X_dataset)
max_samples = total_samples - total_samples % n_sequence

# Reshape
X_dataset = X_dataset[:max_samples]
y_dataset = y_dataset[:max_samples]
X_dataset = X_dataset.reshape(-1, n_sequence, n_features)
y_dataset = y_dataset.reshape(-1, n_sequence, n_classes)

### モデル設計

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D
from keras.regularizers import l2

def createLSTMModel(X_dataset, y_dataset, save_to_path):

    X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.8, random_state=42)
    y_train = y_train[:, -1, :]
    y_test = y_test[:, -1, :]

    # モデル
    model = Sequential([
        Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(n_sequence, n_features)),
        MaxPooling1D(pool_size=2),
        LSTM(512, return_sequences=False, kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(n_classes, activation='softmax')
    ])

    # モデルのコンパイル
    model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

    # モデルチェックポイントのコールバック
    cp_callback = tf.keras.callbacks.ModelCheckpoint(save_to_path, verbose=1, save_weights_only=False)

    # 早期打ち切り用コールバック
    es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

    # モデルのサマリー表示
    #model.summary()

    # 訓練
    model.fit(
        X_train,
        y_train,
        epochs=1000,
        batch_size=128,
        validation_data=(X_test, y_test),
        callbacks=[cp_callback, es_callback],
        verbose=0
    )

    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: {:.2f}%".format(accuracy * 100))

    return model


# モデルを変換(量子化)
def quantizeModel(model, save_to_path):

    # model = tf.keras.models.load_model(model_save_path)

    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

    # TensorFlow Liteでサポートされていない操作を含むモデルに対応するための設定
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,tf.lite.OpsSet.SELECT_TF_OPS]

    converter._experimental_lower_tensor_list_ops = False

    tflite_quantized_model = converter.convert()

    open(save_to_path, 'wb').write(tflite_quantized_model)

## 単発 - モデル作成処理

### モデル構築→量子化前モデル保存→量子化

In [12]:
# モデル構築
model = createLSTMModel(X_dataset, y_dataset, save_to_path=model_save_path)

# テスト
# predict_result = model.predict(np.array([X_test[0]]))

# モデル保存
model.save(model_save_path, include_optimizer=False)

# モデルを変換(量子化)
quantizeModel(model, save_to_path=tflite_save_path)


Epoch 1: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/palm_model/gesture_classifier.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/palm_model/gesture_classifier.hdf5

Epoch 3: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/palm_model/gesture_classifier.hdf5

Epoch 4: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/palm_model/gesture_classifier.hdf5

Epoch 5: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/palm_model/gesture_classifier.hdf5

Epoch 6: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/palm_model/gesture_classifier.hdf5

Epoch 7: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/palm_model/gesture_classifier.hdf5

Epoch 8: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/palm_model/gesture_classifier.hdf5

Epoch 9: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/palm_model/gesture_classifier.hdf5

Epoch 10: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/palm_model/gesture_classifier.hdf5

Epoch 11: saving model to /content/drive/MyDrive/Colab Noteboo

### 量子化モデル推論

In [16]:
# 推論テスト
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

# 入出力テンソルを取得
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 推論実施の前に、入力データを適切な形状にリシェイプ
input_data = np.array([X_dataset[0]]) #.reshape(1, TIME_STEPS, DIMENSION)

In [17]:
# 推論実施
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[2.56685304e-08 6.87786553e-07 2.58062414e-07 5.45114153e-06
 1.42171075e-07 5.87627198e-07 5.30830912e-05 1.44888500e-05
 2.01435454e-04 1.70422414e-08 2.92989660e-11 5.16261971e-05
 2.20977014e-09 3.97980983e-07 4.62004053e-08 8.70149108e-08
 1.06939122e-04 1.06319793e-01 7.36224247e-05 6.77135006e-07
 4.09502432e-09 5.50721415e-06 3.79420690e-08 1.73621273e-09
 1.95725214e-08 5.51149897e-05 2.82780022e-09 1.08463940e-08
 7.10398581e-07 8.08387099e-08 2.74662941e-08 4.70165196e-06
 6.40222481e-07 1.21574402e-07 2.47180942e-10 4.21632222e-08
 1.17170117e-07 5.91866751e-07 1.79721083e-05 3.64186377e-07
 2.59751687e-06 1.47509198e-07 5.90002514e-07 9.78895400e-07
 1.72289710e-05 2.41696085e-09 9.29912460e-07 1.30179833e-04
 9.66266907e-06 1.80268937e-04 9.54831876e-08 2.61098121e-09
 4.23037500e-06 1.42144705e-08 2.92806044e-06 7.45123216e-08
 1.21986233e-07 4.30186949e-04 1.28871351e-02 9.72559574e-05
 1.78943014e-06 2.51889901e-06 7.77804543e-10 4.77178119e-08
 4.94180881e-07 8.792029

### 入力テンソルの形状確認

In [18]:
# モデルのパス
tflite_model_path = tflite_save_path

# TensorFlow Lite インタープリタの初期化
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# 入力ディテールの取得
input_details = interpreter.get_input_details()
print(input_details[0]['shape'])  # 入力テンソルの形状を表示

[ 1 15 40]
